In [23]:
from learning2read.b04 import *
import learning2read
from importlib import reload
reload(learning2read)
learning2read.reload_all()

In [24]:
File = PathMgr('/tmp2/b06902021/ML/file')
Data = DataMgr('/tmp2/b06902021/ML/data') 

In [27]:
Data("users").shape, Data("isbn").shape

((278858, 3), (226, 4))

In [11]:
from learning2read.preprocessing import TotalDataFrame

df_total = TotalDataFrame.run(
    [Data("btrain")])['output']
df_total.sample(10)

,User-ID,ISBN,Book-Rating
175941,62944a452c,0316693235,9
115789,d4c8de598c,9025404162,9
216735,a55a2ab7c0,1551112388,9
16381,139835acac,0451455029,10
169330,47bdcf3c29,0060167084,8
7773,7007df9b76,0312145500,9
122298,4c7159194e,0553802488,8
141848,8959cb241c,884100066X,7
7097,e747cf6b6b,0452282152,8
85244,a20ed75ef8,0425127729,7


In [12]:
procedure6001 = [
    {
        'class' : 'learning2read.preprocessing.TotalDataFrame',
        'output' : 'df_total',
        'input_data' : ['raw_train', 'raw_implicit', 'raw_test'],
    },
    {
        'class' : 'learning2read.preprocessing.UserRatingSqueeze',
        'output' : 'user_rating',
        'input_data' : 'raw_train',
        'filter_num' : 3,
        'statistics' : ['num','mean','std','skew'],
        'na_policy' : 'median',
    },
    {
        'class' : 'learning2read.preprocessing.BookRatingSqueeze',
        'output' : 'book_rating',
        'input_data' : 'raw_train',
        'filter_num' : 3,
        'statistics' : ['num','mean','std','skew'],
        'na_policy' : 'median',
    },
    {
        'class'  : 'learning2read.preprocessing.UserBookTable',
        'output' : 'df_total_features', # (X,y={-1,0,1,2,...,10})
        'input_data' : ['df_total', 'user_rating', 'book_rating'],
        'na_policy' : None, # should fill it before training
    },
    {
        'class'  : 'learning2read.preprocessing.RowFilter',
        'output' : 'df_train',
        'input_data' : 'df_total_features',
        'func' : r"lambda df : df['Book-Rating']>0",
    }
]

In [13]:
pd.DataFrame(procedure6001).loc[:,['class','input_data','output']]

,class,input_data,output
0,learning2read.preprocessing.TotalDataFrame,"[raw_train, raw_implicit]",df_total
1,learning2read.preprocessing.UserRatingSqueeze,raw_train,user_rating
2,learning2read.preprocessing.BookRatingSqueeze,raw_train,book_rating
3,learning2read.preprocessing.UserBookTable,"[df_total, user_rating, book_rating]",df_total_features
4,learning2read.preprocessing.RowFilter,df_total_features,df_train


In [21]:
from learning2read.proc import Procedure
if File.is_readable("p6001_test"):
    P=load_pickle(File("p6001_test"))
else:
    P=Procedure(procedure6001, verbose=True)
    # input data frames
    P.load_data({
        'raw_train': Data('btrain'),
        'raw_test': Data('btest'),
        'raw_implicit': Data('implicit'),
    })

    # run and save
    P.run()
    save_pickle(File("p6001_test"),P)

P.input_from_var('df_train').sample(10)

,Book-Rating,User-ID_is_short,User-ID_mean,User-ID_num,User-ID_skew,User-ID_std,ISBN_is_short,ISBN_mean,ISBN_num,ISBN_skew,ISBN_std
166444,6,0.0,6.333333,9.0,0.192450,1.224745,0.0,7.879581,191.0,-0.662346,1.667381
6206,2,1.0,7.500000,2.0,-0.256120,1.414214,0.0,6.400000,5.0,-0.505301,3.209361
2184,7,1.0,7.500000,1.0,-0.256120,1.414214,0.0,8.467742,62.0,-0.411110,1.224313
66918,7,1.0,7.500000,2.0,-0.256120,1.414214,0.0,8.056818,88.0,-1.070134,1.556285
50902,9,0.0,8.750000,16.0,-0.309839,1.000000,0.0,7.801653,121.0,-0.961969,1.725398
137630,10,1.0,7.500000,1.0,-0.256120,1.414214,0.0,9.500000,4.0,-1.154701,1.000000
211224,9,0.0,8.645161,186.0,-1.063898,1.036054,1.0,7.666667,1.0,-0.239063,1.527525
43909,8,1.0,7.500000,1.0,-0.256120,1.414214,1.0,7.666667,1.0,-0.239063,1.527525
78163,8,0.0,8.644781,1188.0,-1.661702,1.482302,0.0,7.000000,5.0,-0.256120,1.870829
215792,8,0.0,7.800000,10.0,-0.078717,1.475730,1.0,7.666667,1.0,-0.239063,1.527525
